<font size=6 color='#4a86e8'><b><i>Analyst Exercise 2</i></b></font><p align="right">Calculate scroll rates for the top 10 articles</p>

<i><font size=4 color='#4a86e8'>Import packages</font></i>

In [ ]:
import pandas as pd
import re

<i><font size=4 color='#4a86e8'>Load csv data and preview</font></i>

In [ ]:
dat = pd.read_csv('~/AnalystExercises/Data/scroll_depth.csv')
print(dat)

<i><font size=4 color='#4a86e8'>Since our dataframe doesn't have numeric percentiles, we need to add them</font></i>

In [ ]:
dat.insert(
    2, 'Percentile', [int(re.sub('[^0-9]', '', s))/100 for s in dat['EventAction']]
)

<i><font size=4 color='#4a86e8'>If you want you can preview the dataframe to be sure the percentiles were added correctly</font></i>

In [ ]:
print(dat)

<i><font size=4 color='#4a86e8'>Before we do the analysis, we might want to look for duplicated page paths due to case-sensitivity</font></i>

In [ ]:
dat = dat.assign(
    PagePath = [d.lower() for d in dat['PagePath']]
)

<i><font size=4 color='#4a86e8'>Note that some page paths have duplicated values at the same scroll depth. Is case-sensitivity important?</font></i>

In [ ]:
print(dat[dat.duplicated(subset=['PagePath', 'EventAction'], keep=False)].sort_values(by=['PagePath', 'EventAction']))

<i><font size=4 color='#4a86e8'>For now, I'm going to combine the duplicates</font></i>

In [ ]:
subset = dat[dat.duplicated(subset=['PagePath', 'EventAction'], keep=False)].sort_values(by=['PagePath', 'EventAction'])

deduped = subset.groupby(['PagePath', 'EventAction', 'Percentile']).sum().reset_index()

dat = dat.drop_duplicates(subset=['PagePath', 'EventAction'], keep=False).append(deduped).reset_index(drop=True)

<i><font size=4 color='#4a86e8'>If you want you can preview the dataframe to see the changes<br>(note 3 fewer rows than above)</font></i>

In [ ]:
print(dat)

<i><font size=4 color='#4a86e8'>Define a function to calculate the average scroll rate for each subset of the dataframe
<br>Sum of (# of views per each percentile * the percentile)/total pageviews</font></i>

In [ ]:
def avg_scroll_rate(x):
    return round(sum([(row['Pageviews'] * row['Percentile']) for index, row in x.iterrows()])/sum(x['Pageviews']), 3)

<i><b><font size=4 color='#4a86e8'>Finally, we can group our page paths, apply the scroll rate calculation, and view the results</i></b></font>

In [ ]:
grouped = dat.groupby('PagePath')

results = grouped.apply(avg_scroll_rate).sort_values(ascending=False)

print(results)